In [1]:
import pandas as pd
import numpy as np
import os
import time, datetime
import math
import random
from sklearn import preprocessing
import torch
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import seaborn as sns
from mlxtend.classifier import StackingCVClassifier
from scipy.stats import norm, skew
from scipy import stats
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.impute import SimpleImputer
from scipy.misc import derivative
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import linear_model, datasets
from sklearn.linear_model import Lasso
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import lightgbm
from sklearn import svm
from sklearn.neural_network import MLPRegressor,MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
import random
from sklearn.metrics import roc_auc_score
from scipy import optimize
from scipy import special
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn import preprocessing
import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#2020-11-02 18:51:24.674845
#67884674845
def get_time_stamp(time):
    hh = int(time[11:11+2])
    mm = int(time[14:14+2])
    ii = int(time[17:17+2])
    res = hh*3600+mm*60+ii
    for i in range(20,len(time)):
        res=res*10+ord(time[i])-ord('0')
    return res

In [4]:
train_data_label = pd.read_csv('CCF2021_run_label_Train.csv')
label_p_id = train_data_label['人员编号']
label_c_id = train_data_label['特征码']
answer = {label_p_id[i]:label_c_id[i] for i in range(len(label_p_id))}

In [5]:
# train_data = pd.read_csv('train_feature.csv')
# test_data = pd.read_csv('test_feature.csv')
train_data = pd.read_csv('/content/drive/MyDrive/ccf data/train_feature.csv')
test_data = pd.read_csv('/content/drive/MyDrive/ccf data/test_feature.csv')

In [6]:
train_data.shape,test_data.shape

((398338, 14), (196382, 13))

In [ ]:
'''
cols = train_data.columns.tolist()
cols.remove('c_id')
cols.remove('p_id')
cols.remove('label')
for col in cols:
    Min = min(float(train_data[col].min()),float(test_data[col].min()))
    Max = max(float(train_data[col].max()),float(test_data[col].max()))
    train_data.loc[:,col] = (train_data.loc[:,col]-Min)/(Max-Min)
    test_data.loc[:,col] = (test_data.loc[:,col]-Min)/(Max-Min)
'''

"\ncols = train_data.columns.tolist()\ncols.remove('c_id')\ncols.remove('p_id')\ncols.remove('label')\nfor col in cols:\n    Min = min(float(train_data[col].min()),float(test_data[col].min()))\n    Max = max(float(train_data[col].max()),float(test_data[col].max()))\n    train_data.loc[:,col] = (train_data.loc[:,col]-Min)/(Max-Min)\n    test_data.loc[:,col] = (test_data.loc[:,col]-Min)/(Max-Min)\n"

In [7]:
train_data

,c_id,p_id,pp_number,average_det_time,cid_number,unique_pp_number,unique_average_det_time,unique_lcs,p_number,pid_number,p_speed,c_speed,dis_speed,label
0,CY1r8Eov,P0099,0.8,3.437451e+07,558,0,1.000000e+20,2,8,10,8.764114e-10,1.264389e-09,3.879778e-10,0
1,CjD8XlTz,P0099,0.8,3.266357e+07,573,0,1.000000e+20,2,8,10,8.764114e-10,1.103917e-09,2.275057e-10,0
2,C1b6QpCZ,P0099,0.8,2.081857e+08,538,0,1.000000e+20,2,8,10,8.764114e-10,5.378414e-10,3.385700e-10,0
3,C1uhR4YE,P0099,0.8,1.658634e+08,543,0,1.000000e+20,2,8,10,8.764114e-10,1.134907e-09,2.584956e-10,0
4,CwhkYNEx,P0099,0.8,2.051248e+08,545,0,1.000000e+20,2,8,10,8.764114e-10,1.652945e-09,7.765340e-10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
398333,C1mIwd2h,P1967,1.0,8.694088e+09,21,2,7.092226e+09,2,9,9,2.409650e-10,9.620614e-10,7.210963e-10,0
398334,C3LHAtmI,P1967,1.0,6.299373e+09,21,2,2.120187e+10,2,9,9,2.409650e-10,6.214749e-10,3.805099e-10,0
398335,CxHgGpZa,P1967,1.0,1.883359e+10,21,2,1.826556e+10,2,9,9,2.409650e-10,3.818395e-10,1.408745e-10,0
398336,CSlXVwej,P1967,1.0,7.148984e+09,21,2,2.002582e+10,2,9,9,2.409650e-10,7.984698e-10,5.575048e-10,0


In [8]:
test_data

,c_id,p_id,pp_number,average_det_time,cid_number,unique_pp_number,unique_average_det_time,unique_lcs,p_number,pid_number,p_speed,c_speed,dis_speed
0,CbsTmlCa,P0021,1.000000,3.168358e+07,554,4,2.511770e+08,4,10,10,3.664351e-10,1.960694e-09,1.594258e-09
1,CNgGnr1x,P0021,0.900000,3.725253e+07,551,3,3.441018e+08,4,9,10,3.664351e-10,2.562120e-09,2.195685e-09
2,CL0ldUEC,P0021,0.900000,4.142315e+07,554,3,4.301807e+08,4,9,10,3.664351e-10,1.398465e-09,1.032030e-09
3,Cg4mji7e,P0021,0.900000,4.176179e+07,541,3,2.823182e+08,4,9,10,3.664351e-10,1.427462e-09,1.061027e-09
4,CEiaSN40,P0021,0.900000,2.576683e+08,543,3,5.605908e+09,4,9,10,3.664351e-10,2.047915e-09,1.681479e-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...
196377,CV9ngSKy,P0838,0.857143,1.679306e+09,19,1,4.588114e+08,1,6,7,2.142635e-10,1.537771e-10,6.048637e-11
196378,ClU4aBYz,P0838,1.000000,6.296622e+09,19,2,1.944902e+10,2,7,7,2.142635e-10,2.376588e-10,2.339536e-11
196379,CZu7nAtb,P0838,1.000000,1.692636e+10,20,2,9.038338e+09,2,7,7,2.142635e-10,2.802341e-10,6.597060e-11
196380,CaJvM9G1,P0838,1.000000,1.476212e+09,48,2,2.048379e+09,2,7,7,2.142635e-10,9.899794e-10,7.757159e-10


In [9]:
p_ids = []
def init(data):
    global p_ids
    p_ids = data['p_id'].unique().tolist()

In [10]:
init(train_data)

In [11]:
cross_size = 5
def cross_val(model):
    global cross_val,p_ids
    acc = 0.0
    for cross in range(cross_size):
        train_p_ids = set(random.sample(p_ids,k=int(len(p_ids)*0.8)))
        test_p_ids = set()
        train_index = []
        test_index = []
        for p_id in p_ids:
            if p_id not in train_p_ids:
                test_p_ids.add(p_id)
        train_index = train_data[train_data['p_id'].isin(train_p_ids)].index.tolist()
        test_index = train_data[train_data['p_id'].isin(test_p_ids)].index.tolist()
        small_train_data = train_data.iloc[train_index].drop(['c_id','p_id'],axis=1)
        small_test_data = train_data.iloc[test_index]
        test_c_id = small_test_data['c_id'].tolist()
        test_p_id = small_test_data['p_id'].tolist()
        small_test_data.drop(['c_id','p_id'],axis=1,inplace=True)
        model.fit(small_train_data.drop(['label'],axis=1),small_train_data['label'])
        predict_proba_y = model.predict_proba(small_test_data.drop(['label'],axis=1))[:,1:2]
        tmp_answer = {}
        for i in range(len(test_c_id)):
            proba = predict_proba_y[i][0]
            c_id = test_c_id[i]
            p_id = test_p_id[i]
            if proba > tmp_answer.get(p_id,(-1,-1))[1]:
                tmp_answer[p_id] = (c_id,proba)
        true = 0
        for p_id in tmp_answer:
            if tmp_answer[p_id][0] == answer[p_id]:
                true += 1
        print(true,len(test_p_ids),true/len(test_p_ids))
        acc += true/len(test_p_ids)
    print('acc: ',acc/cross_size)

In [12]:
%%time
lgb=lightgbm.LGBMClassifier( objective='binary',
                        # metric='f1_score',
                        learning_rate=0.01,
                        n_estimators=500,
                        boosting= "gbdt",
                        max_depth=10,
                        num_leaves=500,
                        feature_fraction = 0.9,  
                        bagging_fraction = 0.95,  
                        bagging_freq = 5, 
                        )
cross_val(lgb)

357 400 0.8925
346 400 0.865
358 400 0.895
350 400 0.875
356 400 0.89
acc:  0.8835
CPU times: user 4min 14s, sys: 1.62 s, total: 4min 15s
Wall time: 4min 15s


In [13]:
lr = LogisticRegression(class_weight={0:1,1:100},
                        max_iter=5000,
                        random_state=2018)
cross_val(lr)

319 400 0.7975
329 400 0.8225
332 400 0.83
320 400 0.8
330 400 0.825
acc:  0.8150000000000001


In [ ]:
dtc = DecisionTreeClassifier(max_depth=1,
                            class_weight={0:1,1:1})
abc = AdaBoostClassifier(base_estimator = dtc,
                        n_estimators=500,
                        learning_rate=0.15,
                        random_state=2019)
cross_val(abc)

348 400 0.87
346 400 0.865
347 400 0.8675
334 400 0.835
342 400 0.855
acc:  0.8585


In [14]:
# abc.fit(train_data.drop(['c_id','p_id','label'],axis=1),train_data['label'])
model = lgb 
model.fit(train_data.drop(['c_id','p_id','label'],axis=1),train_data['label'])

LGBMClassifier(bagging_fraction=0.95, bagging_freq=5, boosting='gbdt',
               boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               feature_fraction=0.9, importance_type='split',
               learning_rate=0.01, max_depth=10, min_child_samples=20,
               min_child_weight=0.001, min_split_gain=0.0, n_estimators=500,
               n_jobs=-1, num_leaves=500, objective='binary', random_state=None,
               reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)

In [ ]:
###false

test_c_id = train_data['c_id']
test_p_id = train_data['p_id']
predict_proba_y = model.predict_proba(train_data.drop(['c_id','p_id','label'],axis=1))[:,1:2]
tmp_answer = {}
train_acc = 0
false_c_id = []
false_p_id = []
for i in range(len(test_c_id)):
    proba = predict_proba_y[i][0]
    c_id = test_c_id[i]
    p_id = test_p_id[i]
    if proba > tmp_answer.get(p_id,(-1,-1))[1]:
        tmp_answer[p_id] = (c_id,proba)
test_p_id = list(set(test_p_id))
for p_id in test_p_id:
    if tmp_answer[p_id][0] == answer[p_id]:
        train_acc += 1
    else:
        false_p_id.append(p_id)
        false_c_id.append(tmp_answer[p_id][0])
print(train_acc,train_acc/len(test_p_id))
false = pd.DataFrame({'false_pids':false_p_id,
                      'false_cids':false_c_id})
false.to_csv('false.csv',index=False)

In [15]:

test_c_id = test_data['c_id']
test_p_id = test_data['p_id']
predict_proba_y = model.predict_proba(test_data.drop(['c_id','p_id'],axis=1))[:,1:2]
tmp_answer = {}
for i in range(len(test_c_id)):
    proba = predict_proba_y[i][0]
    c_id = test_c_id[i]
    p_id = test_p_id[i]
    if proba > tmp_answer.get(p_id,(-1,-1))[1]:
        tmp_answer[p_id] = (c_id,proba)
test_p_id = list(set(test_p_id))
test_c_id = []
for p_id in test_p_id:
    test_c_id.append(tmp_answer.get(p_id,'-1')[0])
submit = pd.DataFrame({'人员编号':test_p_id,
                       '特征码':test_c_id})
submit.to_csv('output.csv',index=False)

In [ ]:
print(pd.DataFrame({
        'column': test_data.columns[2:], 
        'importance': model.booster_.feature_importance(),
    }).sort_values(by='importance',ascending = False))

                    column  importance
4  unique_average_det_time       63574
1         average_det_time       56029
2               cid_number       42806
3         unique_pp_number       16552
0                pp_number       16282
5               unique_lcs       12017


In [ ]:
test_data.columns

Index(['c_id', 'p_id', 'pp_number', 'average_det_time', 'cid_number',
       'unique_pp_number', 'unique_average_det_time', 'unique_lcs'],
      dtype='object')